# Algorithmen



In [ ]:
%run ./Muehle_Logic.ipynb


In [ ]:
%run ./Muehle_Utilities.ipynb

In [ ]:
START_STATE = [[9, 9], [          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
            [0, 0, 0, 0, 0, 0, 0, 0], # ring 0
            [0, 0, 0, 0, 0, 0, 0, 0], # ring 1
            [0, 0, 0, 0, 0, 0, 0, 0]  # ring 2
      ]]

def to_tupel(State):
    return ((State[0][0], State[0][1]), (
                (State[1][0][0],State[1][0][1],State[1][0][2],State[1][0][3],State[1][0][4],State[1][0][5],State[1][0][6],State[1][0][7]),
                (State[1][1][0],State[1][1][1],State[1][1][2],State[1][1][3],State[1][1][4],State[1][1][5],State[1][1][6],State[1][1][7]),
                (State[1][2][0],State[1][2][1],State[1][2][2],State[1][2][3],State[1][2][4],State[1][2][5],State[1][2][6],State[1][2][7])
            ))

def to_list(State):
    return [[State[0][0], State[0][1]], [
                [State[1][0][0],State[1][0][1],State[1][0][2],State[1][0][3],State[1][0][4],State[1][0][5],State[1][0][6],State[1][0][7]],
                [State[1][1][0],State[1][1][1],State[1][1][2],State[1][1][3],State[1][1][4],State[1][1][5],State[1][1][6],State[1][1][7]],
                [State[1][2][0],State[1][2][1],State[1][2][2],State[1][2][3],State[1][2][4],State[1][2][5],State[1][2][6],State[1][2][7]]
    ]]

to_tupel(START_STATE)

to_list(to_tupel(START_STATE))

Die Funktion `memoize(f)` aus der "Wissensbasierte Systeme"-Vorlesung nimmt ein Argument f und gibt eine Version dieser Funktion zurück, die alle Ergebnisse der Funktion f zwischenspeichert.

In [ ]:
def memoize(f):
    global Cache
    
    def f_memoized(*args):
        if args in Cache:
            return Cache[args]
        result = f(*args)
        Cache[args] = result
        return result
    
    return f_memoized

## Minimax-Algorithmus

In [ ]:
import random
random.seed(1)

`value_minimax(State, player)`

In [ ]:
@memoize
def value_minimax(State, player):
    State = to_list(State)
    if finished(State):
        return utility(State, player)
    o = opponent(player)
    return max([ -value_minimax(to_tupel(ns), o) for ns in next_states(State, player) ])

`best_move_minimax(State, player)`

In [ ]:
def best_move_minimax(State, player):
    NS        = next_states(State, player)
    bestVal   = value_minimax(to_tupel(State), player)
    BestMoves = [s for s in NS if -value_minimax(to_tupel(s), opponent(player)) == bestVal]
    BestState = random.choice(BestMoves)
    return bestVal, BestState

Die Funktion `minimax(State, Player)` wurde erstellt, um die Funktion `best_move_minimax(State, player)` nach Außen eindeutiger von `alpha_beta_pruning(State, Player)` abzugrenzen.

In [ ]:
def minimax(State, player):
    return(best_move_minimax(State, player))

## Alpha-Beta-Pruning

In [ ]:
Cache = {}

`value_ab(State, player, alpha=-1, beta=1)` 

In [ ]:
def value_ab(State, player, alpha=-1, beta=1):
    global Cache
    #to_tupel
    State = to_tupel(State)
    if State in Cache:
        val, a, b = Cache[State]
        if a <= alpha and beta <= b:
            return val
        else:
            alpha = min(alpha, a)
            beta  = max(beta , b)
            val   = alphaBeta(State, player, alpha, beta)
            Cache[State] = val, alpha, beta
            return val
    else:
        val = alphaBeta(State, player, alpha, beta)
        Cache[State] = val, alpha, beta
        return val

`alphaBeta(State, player, alpha, beta)`

In [ ]:
def alphaBeta(State, player, alpha, beta):
    #tolist 
    State = to_list(State)
    if finished(State):
        return utility(State, player)
    val = alpha
    for ns in next_states(State, player):
        val = max(val, -value_ab(ns, opponent(player), -beta, -alpha))
        if val >= beta:
            return val
        alpha = max(val, alpha)
    return val

In [ ]:
def best_move_ab(State, player):
    NS        = next_states(State, player)
    bestVal   = value_ab(State, player)
    BestMoves = [s for s in NS if -value_ab(s, opponent(player)) == bestVal]
    BestState = random.choice(BestMoves)
    return bestVal, BestState

Die Funktion `alpha_beta_pruning(State, Player)` wurde erstellt, um die Funktion `best_move(State, player)` nach Außen eindeutiger von `minimax(State, Player)` abzugrenzen.

In [ ]:
def alpha_beta_pruning(State, player):
    return(best_move_ab(State, player))

In [ ]:
alpha_beta_pruning(START_STATE, 1) #maximale Suchtiefe => muss begrenzt werden!!! => heuristik!!!

In [ ]:
minimax(START_STATE, 1) #funtioniert aktuell nicht